# analyze_expenses



## install/import libs and settings

In [1]:
! pip install pandas 
# ! pip install fuzzywuzzy
# ! pip Install python-Levenshtein
! pip install scikit-learn 



In [2]:
import os

# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

from sklearn import svm


import pandas as pd
# pandas settings
# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

from IPython.display import display, HTML

# DIR = os.path.dirname(os.path.realpath(__file__))
DIR = os.path.abspath('')
DIR_CSV = os.path.join(DIR,'csv')
# ^ this is where you'll put csv files from your bank's website
DIR_REPORTS = os.path.join(DIR,'reports')
# ^ reports will be saved out here


print('DIR: ', DIR)
print('DIR_CSV: ', DIR_CSV)
print('DIR_REPORTS: ', DIR_REPORTS)


DIR:  c:\Users\JGarza\GitHub\analyze_expenses
DIR_CSV:  c:\Users\JGarza\GitHub\analyze_expenses\csv
DIR_REPORTS:  c:\Users\JGarza\GitHub\analyze_expenses\reports


## clean csv files
my bank adds extra lines to the top of the csv files, the code below removes the extra lines.

In [3]:
# clean csv files

for file in os.listdir(DIR_CSV):
    
    new_file = os.path.join(DIR_CSV,file)
    # print(new_file)

    rewrite = False

    lines = []
    with open(new_file,'r') as f:
        lines = f.readlines()

        remove_rows = []
        for index,l in enumerate(lines):
            # print(index,l)
            if l == 'Date,Description,Amount,Running Bal.\n':
                break
            else:
                remove_rows.append(index)

        for rr in remove_rows:
            del lines[0]
            rewrite = True
    
    if rewrite:
        with open(new_file,'w') as f:
            f.writelines(lines)




In [4]:
def compile_csv():
    '''
    combines/compiles all csv files
    '''
    df = pd.DataFrame()
    error_lists = []
    for file in os.listdir(DIR_CSV):
        try:
            newfile = os.path.join(DIR_CSV,file)
            df = pd.concat([df, pd.read_csv(newfile)], axis=0)
        except Exception as e:
            error_lists.append('error:, ' + file + ', ' + str(e))
            pass

    # remove duplicates
    df = df.drop_duplicates()
    df=df.reset_index(drop=True)

    return df, error_lists

df, errors = compile_csv()



## data check 
check the data and make sure you got everything

In [5]:
# print out some info!

print('rows: ', len(df))
if (len(errors) > 0):
    print('errors:')
    print(*errors,sep='\n')

print('\n***colummns***')
display(df.columns)
print('\n***types***')
display(df.dtypes)

print('\n***top 20***')
display(df.head(20))

# display(df)

rows:  172

***colummns***


Index(['Date', 'Description', 'Amount', 'Running Bal.'], dtype='object')


***types***


Date            object
Description     object
Amount          object
Running Bal.    object
dtype: object


***top 20***


,Date,Description,Amount,Running Bal.
0,02/15/2022,Beginning balance as of 02/15/2022,NaN,"17,435.86"
1,02/15/2022,"BANK OF AMERICA, DES:BANK OF AM ID:XXXXXXXXXX0...","8,368.91","25,804.77"
2,02/15/2022,"BANK OF AMERICA, DES:BANK OF AM ID:XXXXXXXXXX0...",394.94,"26,199.71"
3,02/17/2022,MCDONALD'S M2257 OF CA 02/16 PURCHASE CHATSWOR...,-11.81,"26,187.90"
4,02/18/2022,PAYPAL DES:TRANSFER ID:XXXXX43033560 INDN:JUST...,250.14,"26,438.04"
5,02/18/2022,Netflix.com 02/17 PURCHASE netflix.com CA,-13.99,"26,424.05"
6,02/18/2022,MCDONALD'S M2257 OF CA 02/17 PURCHASE CHATSWOR...,-11.81,"26,412.24"
7,02/18/2022,SMART AND FINA 02/18 PURCHASE CHATSWORTH CA,-37.09,"26,375.15"
8,02/18/2022,VONS #1671 02/18 PURCHASE CHATSWORTH CA,-12.97,"26,362.18"
9,02/22/2022,STARBUCKS STORE 05341 02/17 PURCHASE CHATSWORT...,-15.95,"26,346.23"


## modifying columns
add a few new columns

In [20]:
# lets change some of the columns



df['Running Bal.'] = df['Running Bal.'].fillna(0)
df['Running Bal.'] = df['Running Bal.'].replace('[,|$]','', regex=True)
df['Running Bal.'] = df['Running Bal.'].astype(float)

df['Amount'] = df['Amount'].fillna(0)
df['Amount'] = df['Amount'].replace('[,|$]','', regex=True)
df['Amount'] = df['Amount'].astype(float)

df['month'] = df['Date'].str[0:2].astype(int)
df['day'] = df['Date'].str[3:5].astype(int)
df['year'] = df['Date'].str[6:10].astype(int)
df['yearmonth'] = 100 * df['year'] + df['month']

df['desc'] = df['Description']
df['desc'] = df['desc'].replace('(,|:|ID:|INDN:|DES)',' ', regex=True)
# df['desc'] = df['desc'].replace('(\d|\/|#|*|\'|JUSTIN|GARZA|Justin|Garza|X{3,50}|PURCHASE|(\s)CA(\s|$)|CHATSWORTH|\.com|\.COM|BILL|\.COMBILL|\sCO\s|OF|THE|\s[A-Z]\s)','', regex=True)
df['desc'] = df['desc'].replace('(\d|\/|#|\*|\'|JUSTIN|GARZA|Justin|Garza|X{3,50}|PURCHASE|(\s)CA(\s|$)|CHATSWORTH|\.com|\.COM|BILL|\.COMBILL|\sCO\s|OF|THE|\s[A-Z]\s)','', regex=True)
df['desc'] = df['desc'].replace('\s\s*',' ', regex=True)

#might not need this one
df['desc_list'] = df['desc'].str.split()


display(df.head(10))
print(df.dtypes)



,Date,Description,Amount,Running Bal.,month,day,year,yearmonth,desc,desc_list
0,02/15/2022,Beginning balance as of 02/15/2022,0.00,17435.86,2,15,2022,202202,Beginning balance as of,"[Beginning, balance, as, of]"
1,02/15/2022,"BANK OF AMERICA, DES:BANK OF AM ID:XXXXXXXXXX0...",8368.91,25804.77,2,15,2022,202202,BANK AMERICA BANK AM PPD,"[BANK, AMERICA, BANK, AM, PPD]"
2,02/15/2022,"BANK OF AMERICA, DES:BANK OF AM ID:XXXXXXXXXX0...",394.94,26199.71,2,15,2022,202202,BANK AMERICA BANK AM PPD,"[BANK, AMERICA, BANK, AM, PPD]"
3,02/17/2022,MCDONALD'S M2257 OF CA 02/16 PURCHASE CHATSWOR...,-11.81,26187.90,2,17,2022,202202,MCDONALDS M,"[MCDONALDS, M]"
4,02/18/2022,PAYPAL DES:TRANSFER ID:XXXXX43033560 INDN:JUST...,250.14,26438.04,2,18,2022,202202,PAYPAL TRANSFER PAYPALSD PPD,"[PAYPAL, TRANSFER, PAYPALSD, PPD]"
5,02/18/2022,Netflix.com 02/17 PURCHASE netflix.com CA,-13.99,26424.05,2,18,2022,202202,Netflix netflix,"[Netflix, netflix]"
6,02/18/2022,MCDONALD'S M2257 OF CA 02/17 PURCHASE CHATSWOR...,-11.81,26412.24,2,18,2022,202202,MCDONALDS M,"[MCDONALDS, M]"
7,02/18/2022,SMART AND FINA 02/18 PURCHASE CHATSWORTH CA,-37.09,26375.15,2,18,2022,202202,SMART AND FINA,"[SMART, AND, FINA]"
8,02/18/2022,VONS #1671 02/18 PURCHASE CHATSWORTH CA,-12.97,26362.18,2,18,2022,202202,VONS,[VONS]
9,02/22/2022,STARBUCKS STORE 05341 02/17 PURCHASE CHATSWORT...,-15.95,26346.23,2,22,2022,202202,STARBUCKS STORE,"[STARBUCKS, STORE]"


Date             object
Description      object
Amount          float64
Running Bal.    float64
month             int32
day               int32
year              int32
yearmonth         int32
desc             object
desc_list        object
dtype: object


In [30]:
# display(df)

import numpy as np
dfg = df.pivot_table(index=['yearmonth','desc'],values=['Amount'],aggfunc=np.sum)
display(dfg)


Amount
yearmonth desc                                            
202112    BANK AMERICA BANK AM PPD                 2068.88
          Beginning balance as of                     0.00
          CICEKS CHICKEN                            -35.84
          EARLS DONUT                                -3.00
          Elastum_Simplex s@simplex                 -50.00
          JERSEY MIKES PORTER RANCH                 -20.94
          PAYRANGE MOBILE MOBILE XX                 -25.00
          RITE AID                                  -32.61
          SMART AND FINA                            -49.65
          SPOTIFY -- NY                              -9.99
          WM SUPERC Wal- PORTER RANCH                -1.93
202201    AMAZONKJTI AM AMZN WA                     -66.14
          AMZN MKTP USJO AMZN WA                   -114.77
          AMZN MKTP USJYDO AMZN WA                  -49.60
          APPLE CASH PMNT SENT --                   -50.00
          BANK AMERICA BANK AM PPD                 4170.67
          BANK AMERICA PPD_US PPD                    70.00
          BEST BUY MHT REFUND NORTHRIDGE             21.89
          BEST BUY NORTHRIDGE                       -21.89
          BURGER KING                               -67.15
          Beginning balance as of                     0.00
          CHEVRONTHREE PORTER RANCH                 -50.00
          Counter Credit                           1000.00
          EARLS DONUT                                -5.25
          Interest Earned                             0.15
          LEMONADE I LEMONADE LEMONADE NY           -48.25
          LITTLE CAESARS Q                          -32.41
          MASONGROVE HOMEO MASONGROVE FT HOA WEB   -518.00
          MCDONALDS M                               -17.91
          MERCURY INS PAYMENT A PPD                 -83.27
          NICS BURGERS AND BOWLS                    -15.82
          Netflix netflix                           -13.99
          PAYPAL TRANSFER PAYPALSD PPD              243.14
          POPEYES                                   -27.36
          RITE AID                                  -12.34
          SDCSEVIER COUNTY ELECT -XX TN            -265.35
          SILVER PANDA                              -26.17
          SMART AND FINA                           -238.52
          SO CAL GAS PAID SCGC WEB                  -32.50
          SPOTIFY -- NY                              -9.99
          SQ EPIC TACO Sun Valley                   -27.13
          SUNTRUST OLB MTGPMT WEB                 -3182.31
          Subway Chatsworth                         -22.46
          TESEVIERUTILITY -- TN                     -41.48
          TMOBILEAUTO PAY -- WA                     -70.00
          The Hungry Ninja                          -16.82
          VONS                                      -47.62
202202     UPS STORE                                 -1.00
          AMAZONIKDG AM AMZN WA                     -16.19
          AMZN Mktp USIXD Amznbill WA               -61.32
          ARCO                                      -67.14
          BANK AMERICA BANK AM PPD                17492.97
          BURGER KING                               -19.68
          Beginning balance as of                     0.00
          CITY LA DWP --                           -227.00
          EARLS DONUT                                -7.75
          HABIT PORTER RANCH PORTER RANCH           -13.55
          INOUT BURGER PORTER RANCH                 -27.48
          INTUIT TURBOTAX CL.INTUIT                -128.00
          Interest Earned                             0.14
          LAL MIRCH AGOURA HILLS                    -30.23
          LEMONADE I LEMONADE LEMONADE NY           -48.25
          LITTLE CAESARS Q                           -8.81
          MASONGROVE HOMEO MASONGROVE FT HOA WEB   -518.00
          MCDONALDS M                               -67.54
          MERCURY INS PAYMENT A PPD                 -83.27
          MGD KOREAN BBQ NORTHRIDGE     

## trying fuzzy
❌❌❌❌❌❌ 

In [7]:
'''
trying fuzzy... not really what i'm looking for 
'''
print(len(list(df.columns)))
print(len(df))

# for i in range(len(df)):
#     # print(df[i:i+1]['desc'])
#     for j in range(len(df)):
#     #     print(df[j:j+1]['desc'])
#         a_desc = df.loc[i,'desc']
#         b_desc = df.loc[j,'desc']
#         print(type(a_desc),a_desc)
#         print(type(b_desc),b_desc)
#         score = fuzz.partial_ratio(a_desc,b_desc)
#         print(score)
#         print('***')
    


10
172


## try SVM from scikit

https://scikit-learn.org/stable/modules/svm.html#classification  
or  
https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a  
or   
https://github.com/javedsha/text-classification/blob/master/Text%2BClassification%2Busing%2Bpython%2C%2Bscikit%2Band%2Bnltk.py  

In [8]:
'''
needed if we use an AI
'''

# df['desc_list'] = df['desc'].str.split(" ")
# print(df[['desc','desc_list']].head(10))

'\nneeded if we use an AI\n'